In [1]:
import pymysql
import requests

下載分析pm2.5 opendata資料

In [8]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [9]:
resp=requests.get(url,verify=False)
resp

c:\Users\USER\Desktop\Django_Lennox\MYSQL\pm25_tomysql\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [10]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '14',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '4',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '11',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '3',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '7',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '6',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '',
  'datacreationdate': '2025-08-27 20:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '10',
  'datacreationdate'

In [39]:
len(datas)

1000

In [40]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '14',
 'datacreationdate': '2025-08-27 20:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [6]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""

print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [2]:
conn=pymysql.connect(
host="mysql-16541887-lennox0925-pm25.j.aivencloud.com",
user="avnadmin",
password="AVNS_LuyFbfSDlFVuJn7gHUY",
port=13848,
database="defaultdb"
)

cursor=conn.cursor()
cursor

In [7]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [26]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"
data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '14', '2025-08-27 20:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',14,'2025-08-27 20:00','μg/m3')"

In [27]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入
- 加上 ignore

In [11]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

#移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]

cursor.executemany(sqlstr,values)
conn.commit()

In [12]:
conn.close()

In [30]:
import pandas as pd 

In [31]:
cursor.execute("select * from pm25")

datas=cursor.fetchall()
datas

InterfaceError: (0, '')

In [ ]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])

df[["site","datetime"]].drop_duplicates()

,site,datetime
0,員林,2025-08-20 19:00:00
1,大城,2025-08-20 19:00:00
2,富貴角,2025-08-20 19:00:00
3,麥寮,2025-08-20 19:00:00
4,關山,2025-08-20 19:00:00
...,...,...
546,新莊,2025-08-20 20:00:00
547,板橋,2025-08-20 20:00:00
548,土城,2025-08-20 20:00:00
549,汐止,2025-08-20 20:00:00


### 取得各縣市站點的平均值

In [5]:
cursor.execute("select avg(pm25) from pm25;")
float(cursor.fetchone()[0])

7.6538

In [ ]:
sqlstr="""
select county,round(avg(pm25),2) from pm25 group by county ;
"""

cursor.execute(sqlstr)
cursor.fetchall()

In [ ]:
conn.close()

### 取得單一縣市的站點數值

In [ ]:
county="新北市"

# 取得最新資料
sqlstr='''
select site,pm25 ,datacreationdate from pm25 
where county=%s;
'''
cursor.execute(sqlstr,(county,))
datas=cursor.fetchall()
datas

(('富貴角', 4, datetime.datetime(2025, 8, 27, 20, 0)),
 ('永和', 3, datetime.datetime(2025, 8, 27, 20, 0)),
 ('三重', 7, datetime.datetime(2025, 8, 27, 20, 0)),
 ('淡水', 7, datetime.datetime(2025, 8, 27, 20, 0)),
 ('林口', 5, datetime.datetime(2025, 8, 27, 20, 0)),
 ('菜寮', 5, datetime.datetime(2025, 8, 27, 20, 0)),
 ('新莊', 3, datetime.datetime(2025, 8, 27, 20, 0)),
 ('板橋', 3, datetime.datetime(2025, 8, 27, 20, 0)),
 ('土城', 3, datetime.datetime(2025, 8, 27, 20, 0)),
 ('新店', 2, datetime.datetime(2025, 8, 27, 20, 0)),
 ('汐止', 6, datetime.datetime(2025, 8, 27, 20, 0)),
 ('富貴角', 4, datetime.datetime(2025, 8, 27, 19, 0)),
 ('永和', 6, datetime.datetime(2025, 8, 27, 19, 0)),
 ('三重', 10, datetime.datetime(2025, 8, 27, 19, 0)),
 ('淡水', 12, datetime.datetime(2025, 8, 27, 19, 0)),
 ('林口', 6, datetime.datetime(2025, 8, 27, 19, 0)),
 ('菜寮', 5, datetime.datetime(2025, 8, 27, 19, 0)),
 ('新莊', 7, datetime.datetime(2025, 8, 27, 19, 0)),
 ('板橋', 3, datetime.datetime(2025, 8, 27, 19, 0)),
 ('土城', 6, datetime.datetim